In [1]:
import pandas as pd
import numpy as np
import os

# ==============================================================================
# 1. CẤU HÌNH ĐƯỜNG DẪN (Sửa lại cho đúng input của bạn)
# ==============================================================================
INPUT_PATHS = {
    "ViMedNLI": "/kaggle/input/nli-halu/NLI/ViMedNLI/csv_processed/",
    "ViANLI":   "/kaggle/input/nli-halu/NLI/ViANLI/",
    "ViNLI":    "/kaggle/input/nli-halu/NLI/ViNLI/csv/"
}

OUTPUT_DIR = "./processed_data_3labels"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==============================================================================
# 2. HÀM CHUYỂN ĐỔI NHÃN (CORE LOGIC - 3 LABELS)
# ==============================================================================
def standardize_label(val):
    """
    Hàm chuẩn hóa nhãn về 3 loại theo đúng slide báo cáo:
    - Entailment (0)    -> Entailment
    - Contradiction (1) -> Intrinsic-Hal
    - Neutral (2)       -> Extrinsic-Hal
    """
    # Chuyển về string chữ thường và xóa khoảng trắng
    s = str(val).lower().strip()
    
    # --- Nhóm 1: Entailment (Tin cậy) ---
    if s == '0' or s == '0.0' or 'entailment' in s:
        return "Entailment"
    
    # --- Nhóm 2: Contradiction (Mâu thuẫn nội tại) ---
    # Lưu ý: Trong dataset gốc thường 2 là Contradiction
    if s == '1' or s == '1.0' or 'contradiction' in s:
        return "Intrinsic-Hal"
    
    # --- Nhóm 3: Neutral (Bịa đặt/Ngoại lai) ---
    # Lưu ý: Trong dataset gốc thường 1 là Neutral
    if s == '2' or s == '2.0' or 'neutral' in s:
        return "Extrinsic-Hal"
    
    # Trường hợp lỗi/không xác định
    return "Unknown"

# ==============================================================================
# 3. HÀM XỬ LÝ FILE
# ==============================================================================
def process_csv(dataset_name, file_type):
    input_file = os.path.join(INPUT_PATHS[dataset_name], f"{file_type}.csv")
    
    if not os.path.exists(input_file):
        print(f"⚠️ Không tìm thấy file: {input_file}")
        return

    print(f"🔄 Đang xử lý {dataset_name} - {file_type}...")
    
    # 1. Đọc dữ liệu
    df = pd.read_csv(input_file)
    
    # 2. Lọc bỏ dữ liệu rác
    initial_len = len(df)
    df.dropna(subset=['sentence1', 'sentence2', 'label'], inplace=True)
    
    # 3. Áp dụng logic chuyển nhãn (Vectorized)
    vectorized_label_func = np.vectorize(standardize_label)
    df['label'] = vectorized_label_func(df['label'])
    
    # 4. Kiểm tra và loại bỏ các dòng Unknown (nếu có)
    unknown_count = len(df[df['label'] == 'Unknown'])
    if unknown_count > 0:
        print(f"   ⚠️ Cảnh báo: Có {unknown_count} dòng nhãn lạ, đã loại bỏ.")
        df = df[df['label'] != 'Unknown']

    # 5. Lưu file mới
    output_subdir = os.path.join(OUTPUT_DIR, dataset_name)
    os.makedirs(output_subdir, exist_ok=True)
    
    output_file = os.path.join(output_subdir, f"{file_type}.csv")
    df.to_csv(output_file, index=False)
    
    print(f"✅ Xong! Giữ lại {len(df)}/{initial_len} dòng. Lưu tại: {output_file}")
    print(f"   Phân bố nhãn: {df['label'].value_counts().to_dict()}\n")

# ==============================================================================
# 4. CHẠY XỬ LÝ CHO TẤT CẢ
# ==============================================================================

datasets = ["ViMedNLI", "ViANLI", "ViNLI"]
file_types = ["train", "dev", "test"]

for ds in datasets:
    for f_type in file_types:
        process_csv(ds, f_type)

print(f"\n🎉 Đã xử lý xong toàn bộ! Dữ liệu mới nằm trong thư mục: {OUTPUT_DIR}")

🔄 Đang xử lý ViMedNLI - train...
✅ Xong! Giữ lại 11232/11232 dòng. Lưu tại: ./processed_data_3labels/ViMedNLI/train.csv
   Phân bố nhãn: {'Entailment': 3744, 'Intrinsic-Hal': 3744, 'Extrinsic-Hal': 3744}

🔄 Đang xử lý ViMedNLI - dev...
✅ Xong! Giữ lại 1395/1395 dòng. Lưu tại: ./processed_data_3labels/ViMedNLI/dev.csv
   Phân bố nhãn: {'Entailment': 465, 'Intrinsic-Hal': 465, 'Extrinsic-Hal': 465}

🔄 Đang xử lý ViMedNLI - test...
✅ Xong! Giữ lại 1422/1422 dòng. Lưu tại: ./processed_data_3labels/ViMedNLI/test.csv
   Phân bố nhãn: {'Entailment': 474, 'Intrinsic-Hal': 474, 'Extrinsic-Hal': 474}

🔄 Đang xử lý ViANLI - train...
✅ Xong! Giữ lại 8012/8012 dòng. Lưu tại: ./processed_data_3labels/ViANLI/train.csv
   Phân bố nhãn: {'Extrinsic-Hal': 2924, 'Entailment': 2615, 'Intrinsic-Hal': 2473}

🔄 Đang xử lý ViANLI - dev...
✅ Xong! Giữ lại 1000/1000 dòng. Lưu tại: ./processed_data_3labels/ViANLI/dev.csv
   Phân bố nhãn: {'Entailment': 334, 'Extrinsic-Hal': 333, 'Intrinsic-Hal': 333}

🔄 Đang xử 